# 뉴럴 스타일 트랜스퍼(Neural Stype Transfer)

케라스 창시자에게 배우는 딥러닝, 프랑소와 숄레, 길벗
* https://github.com/rickiepark/deep-learning-with-python-2nd/blob/main/chapter12_part03_neural-style-transfer.ipynb

### 케라스로 뉴럴 스타일 트랜스퍼 구현하기

**스타일 이미지와 콘텐츠 이미지 준비하기**

In [ ]:
from tensorflow import keras

base_image_path = keras.utils.get_file(
    "about_onlinecampus_img1.png", origin="https://grad.iscu.ac.kr/images/about/about_onlinecampus_img1.png")
style_reference_image_path = keras.utils.get_file(
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

943128/943128 [==============================] - 0s 0us/step


**유틸리티 함수**

In [ ]:
import numpy as np

def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

**사전 훈련된 VGG19 모델을 사용해 특성 추출기 만들기**

In [ ]:
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

80134624/80134624 [==============================] - 1s 0us/step


**콘텐츠 손실**

In [ ]:
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

**스타일 손실**

In [ ]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

**총 변위 손실**

In [ ]:
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :]
    )
    b = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

**최소화할 최종 손실 정의하기**

In [ ]:
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

**경사 하강법 단계 설정하기**

In [ ]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 10 == 0:
        print(f"{i}번째 반복: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)

10번째 반복: loss=27014.72
20번째 반복: loss=17268.84
30번째 반복: loss=14628.40
40번째 반복: loss=13134.40
50번째 반복: loss=12105.21
60번째 반복: loss=11329.20
70번째 반복: loss=10712.32
80번째 반복: loss=10205.16
90번째 반복: loss=9779.09
100번째 반복: loss=9415.63
110번째 반복: loss=9101.85
120번째 반복: loss=8827.91
130번째 반복: loss=8586.39
140번째 반복: loss=8372.11
150번째 반복: loss=8180.41
160번째 반복: loss=8007.98
170번째 반복: loss=7852.12
180번째 반복: loss=7710.62
190번째 반복: loss=7581.59
200번째 반복: loss=7463.48
210번째 반복: loss=7354.94
220번째 반복: loss=7254.81
230번째 반복: loss=7162.28
240번째 반복: loss=7076.56
250번째 반복: loss=6996.84
260번째 반복: loss=6922.56
270번째 반복: loss=6853.26
280번째 반복: loss=6788.41
290번째 반복: loss=6727.66
300번째 반복: loss=6670.65
310번째 반복: loss=6617.05
320번째 반복: loss=6566.48
330번째 반복: loss=6518.77
340번째 반복: loss=6473.69
350번째 반복: loss=6430.99
360번째 반복: loss=6390.50
370번째 반복: loss=6352.09
380번째 반복: loss=6315.53
390번째 반복: loss=6280.72
400번째 반복: loss=6247.52
410번째 반복: loss=6215.83
420번째 반복: loss=6185.56
430번째 반복: loss=6156.60
440번째 반복: lo